In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import xlsxwriter
from itertools import chain
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import glob
import os
import re

import sys
sys.path.append('../utils')
from SharePoint import *
from hlpr import *
from static import *
from format_qa import *
from qa_by_media import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
media = 'social'
platform = 'tw'

SHAREPOINT_DATA_PATH = media_args[media]['sharepoint']['data']
SHAREPOINT_MAPPING_PATH = media_args[media]['sharepoint']['mapping']
SHAREPOINT_QA_PATH = media_args[media]['sharepoint']['qa']
SHAREPOINT_FLAT_PATH = media_args[media]['sharepoint']['flat']
REDSHIFT_METRICS_QUERY = media_args[media]['redshift']['all_metrics']
REDSHIFT_METRICS_FILENAME = media_args[media]['redshift']['metric_filename']
REDSHIFT_METRICS_SHEETNAME = media_args[media]['redshift']['metric_sheetname']
REDSHIFT_QA_QUERY = media_args[media]['redshift']['qa_qry']

**Connect to SharePoint**

In [3]:
app = SharePoint('AmericanExpressUSGABM')

# download data
# app.list_contents(SHAREPOINT_DATA_PATH)
# app.download_files(DATA_PATH)

# ancillary_data(app, media)

# # download floodlight mapping
# app.list_contents(SHAREPOINT_MAPPING_PATH)
# app.download_files(ASSETS_PATH)

Connected to SharePoint: https://interpublic.sharepoint.com/sites/AmericanExpressUSGABM


In [8]:
# import qa
df_qa = pd.read_csv(glob.glob(os.path.join('../misc', '*qa*'))[0])

In [18]:
pth = glob.glob(os.path.join(ASSETS_PATH, "*Social_Metrics*"))[0]
df_metrics = pd.read_excel(pth, sheet_name='tw_conversions')
floodlight = list(df_metrics['floodlight_name'].unique())
universal_campaign = list(df_metrics['universal_campaign'].unique())

In [17]:
df_metrics

,universal_campaign,activity,activity_id,floodlight_name
0,21_Q1_USA_GA_TW_LINK_Dine Small - Evergreen,Resy Takeout,10669167,Resy : Resy Takeout: Click-through Conversion ...
1,21_Q1_USA_GA_TW_LINK_Dine Small - Evergreen,Resy Takeout,10669167,Resy : Resy Takeout: View-through Conversion E...
2,21_Q1_USA_GA_TW_LINK_Dine Small - Evergreen,Shop Small Page,4882628,2017 GABM GCP : Shop Small Page: View-through ...
3,21_Q1_USA_GA_TW_LINK_Dine Small - Evergreen,Shop Small Page,4882628,2017 GABM GCP : Shop Small Page: Click-through...
4,21_Q1_USA_GA_TW_LINK_Dine Small - Takeout Tuesday,Resy Takeout,10669167,Resy : Resy Takeout: Click-through Conversion ...
5,21_Q1_USA_GA_TW_LINK_Dine Small - Takeout Tuesday,Resy Takeout,10669167,Resy : Resy Takeout: View-through Conversion E...
6,21_Q1_USA_GA_TW_LINK_Dine Small - Takeout Tuesday,Shop Small Page,4882628,2017 GABM GCP : Shop Small Page: View-through ...
7,21_Q1_USA_GA_TW_LINK_Dine Small - Takeout Tuesday,Shop Small Page,4882628,2017 GABM GCP : Shop Small Page: Click-through...


In [30]:
# split
df_qa_split = df_qa.copy()
df_qa_split_0 = df_qa[-df_qa['variable'].isin(floodlight)] # core metrics
df_qa_split_1 = df_qa[df_qa['variable'].isin(floodlight)] # floodlights
df_qa_split_2 = df_qa_split_1[df_qa_split_1['campaign_name'].isin(universal_campaign)] # floodlight and campaing
df_qa = pd.concat([df_qa_split_0, df_qa_split_2], sort=False).reset_index(drop=True)

In [31]:
df_qa

,platform,week,campaign_name,adset,campaign_id,adset_id,variable,value,source
0,tw,2021-02-01,21_Q1_USA_GA_TW_LINK_Dine Small - Takeout Tuesday,MediaTools Audience_In Feed_na_Dine Small_A255...,25251840,31393299,Impressions,4.406470e+05,redshift
1,tw,2021-02-01,21_Q1_USA_GA_TW_LINK_Dine Small - Takeout Tuesday,MediaTools Audience_In Feed_na_Dine Small_A255...,25251840,31393299,Link clicks,8.380000e+02,redshift
2,tw,2021-02-01,21_Q1_USA_GA_TW_LINK_Dine Small - Takeout Tuesday,MediaTools Audience_In Feed_na_Dine Small_A255...,25251840,31393299,Spend,4.680000e+03,redshift
3,tw,2021-02-01,21_Q1_USA_GA_TW_LINK_Dine Small - Takeout Tuesday,MediaTools Audience_In Feed_na_Dine Small_A255...,25251840,31393300,Impressions,3.445760e+05,redshift
4,tw,2021-02-01,21_Q1_USA_GA_TW_LINK_Dine Small - Takeout Tuesday,MediaTools Audience_In Feed_na_Dine Small_A255...,25251840,31393300,Link clicks,7.170000e+02,redshift
5,tw,2021-02-01,21_Q1_USA_GA_TW_LINK_Dine Small - Takeout Tuesday,MediaTools Audience_In Feed_na_Dine Small_A255...,25251840,31393300,Spend,4.680000e+03,redshift
6,tw,2021-02-08,21_Q1_USA_GA_TW_LINK_Dine Small - Evergreen,MediaTools Audience_In Feed_na_Dine Small_A255...,25299944,31457430,Impressions,1.196814e+06,redshift
7,tw,2021-02-08,21_Q1_USA_GA_TW_LINK_Dine Small - Evergreen,MediaTools Audience_In Feed_na_Dine Small_A255...,25299944,31457430,Link clicks,2.825000e+03,redshift
8,tw,2021-02-08,21_Q1_USA_GA_TW_LINK_Dine Small - Evergreen,MediaTools Audience_In Feed_na_Dine Small_A255...,25299944,31457430,Spend,1.169177e+04,redshift
9,tw,2021-02-08,21_Q1_USA_GA_TW_LINK_Dine Small - Evergreen,MediaTools Audience_In Feed_na_Dine Small_A255...,25299944,31457432,Impressions,1.433093e+06,redshift
